In [1]:
from transformers import BertTokenizer
from razdel import sentenize
from models.model_builder import AbsSummarizer
import torch
import numpy as np
import pandas as pd
import tqdm
import json
import pickle
import os

C:\Users\leshanbog\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\leshanbog\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\leshanbog\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\leshanbog\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Futu

In [2]:
DEVICE = 'cuda'

In [3]:
class BertData:
    def __init__(self, bert_model, lower, max_src_tokens, max_tgt_tokens):
        self.max_src_tokens = max_src_tokens
        self.max_tgt_tokens = max_tgt_tokens
        self.tokenizer = BertTokenizer.from_pretrained(bert_model, do_lower_case=lower, do_basic_tokenize=False)
        self.sep_token = '[SEP]'
        self.cls_token = '[CLS]'
        self.pad_token = '[PAD]'
        self.tgt_bos = '[unused1] '
        self.tgt_eos = ' [unused2]'
        self.tgt_sent_split = ' [unused3] '
        self.sep_vid = self.tokenizer.vocab[self.sep_token]
        self.cls_vid = self.tokenizer.vocab[self.cls_token]
        self.pad_vid = self.tokenizer.vocab[self.pad_token]

    def preprocess(self, src, tgt):
        src_txt = [' '.join(s) for s in src]
        text = ' {} {} '.format(self.sep_token, self.cls_token).join(src_txt)
        src_tokens = self.tokenizer.tokenize(text)[:self.max_src_tokens]
        src_tokens.insert(0, self.cls_token)
        src_tokens.append(self.sep_token)
        src_indices = self.tokenizer.convert_tokens_to_ids(src_tokens)

        _segs = [-1] + [i for i, t in enumerate(src_indices) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if i % 2 == 0:
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]

        return src_indices, segments_ids

In [4]:
def doc2bert(text):
    src = [s.text.lower().split() for s in sentenize(text)]
    src_indices, segments_ids = bert_data.preprocess(src, '')
    return { "src": src_indices, "segs": segments_ids }

def doc2vec(text, model, mode='MeanSum'):
    doc_bert = doc2bert(text)
    
    src = torch.tensor([doc_bert['src']])
    segs = torch.tensor([doc_bert['segs']])
    mask_src = ~(src == 0)
    
    output = model.bert(src.to(DEVICE), segs.to(DEVICE), mask_src.to(DEVICE))
    
    if mode == 'FirstCLS':
        return output[0][0]
    elif mode == 'MeanSum':
        return output[0].mean(0)
    else:
        raise Exception('Wrong mode')

In [5]:
checkpoint = torch.load(r'C:\Users\leshanbog\Documents\model\model_step_15000.pt',
                        map_location=lambda storage, loc: storage)


In [7]:
args = lambda a: b

args.model_path = r'C:\Users\leshanbog\Documents\model\bert\rubert_cased_L-12_H-768_A-12_pt'
args.large = False
args.temp_dir = 'temp'
args.finetune_bert = False
args.encoder = 'bert'
args.max_pos = 256
args.dec_layers = 6
args.share_emb = False
args.dec_hidden_size = 768
args.dec_heads = 8
args.dec_ff_size = 2048
args.dec_dropout = 0.2
args.use_bert_emb = False

bert_data = BertData(args.model_path, True, 510, 128)

BertSumAbs = AbsSummarizer(args, DEVICE, checkpoint)
BertSumAbs.eval()

AbsSummarizer(
  (bert): Bert(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): BertLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): BertLayerNorm()
              

In [11]:
import os
from tqdm import trange

class TelegramPredictionsDataset:
    def __init__(self, jsonl_path):
        data = pd.read_json(jsonl_path, lines=True)
        self.texts = []
        self.titles = []
        self.target_vectors = []
        for i in trange(len(data)):
            text = data.iloc[i]['text'].lower().replace('\xa0', ' ').replace('\n', ' ').strip()
            title = data.iloc[i]['title'].lower()

            if not text or not title or text.count(' ') < 3 or title.count(' ') < 3:
                continue
                
            self.texts.append(text)
            self.titles.append(title)
            self.target_vectors.append(doc2vec(text + " " + title, BertSumAbs, mode="FirstCLS"))
            
        self.target_vectors = torch.FloatTensor(self.target_vectors)
        
    def __len__(self):
        return len(self.dataset)
        
    def __getitem__(self, idx):
        return self.texts[idx], self.titles[idx], self.target_vectors[idx]

## Constructing dataset

In [10]:
CHINK_SIZE = 1024

data = pd.read_json(r"C:\Users\leshanbog\Documents\dataset\ru_tg_1101_0510.jsonl", encoding='utf-8',
                    lines=True, chunksize=CHINK_SIZE)

In [11]:
for el in tqdm.tqdm(data, total=450000 // CHINK_SIZE):
    with open('vectors.npy', 'ab') as fvecs, open('text.jsonl', 'a', encoding='utf-8') as ft:
        for j in range(CHINK_SIZE):
            text = el.iloc[j]["text"].lower().replace('\xa0', ' ').replace('\n', ' ').strip()
            title = el.iloc[j]["title"].lower()

            if not text or not title or text.count(' ') < 8 or title.count(' ') < 3:
                continue

            ft.write(json.dumps({"text": text, "title": title}) + "\n")

            vec = doc2vec(text + " " + title, BertSumAbs, mode="FirstCLS")
            vec = vec.cpu().detach().numpy()
            np.save(fvecs, vec)

474it [9:12:48, 68.45s/it]                                                                                             

KeyboardInterrupt: 

### Reading dataset

In [15]:
with open("vectors.npy", "rb") as fh:
    fsz = os.fstat(fh.fileno()).st_size
    out = np.load(fh).reshape(1, -1)
    a = 0
    while fh.tell() < fsz:
        out = np.concatenate((out, np.load(fh).reshape(1, -1)), axis=0)
        a += 1
        if a == 20:
            break
        
out.shape

(21, 768)

In [13]:
b = pd.read_json("text.jsonl",encoding='utf-8',
                    lines=True, chunksize=CHINK_SIZE)
b

### Encoder with pretrained FastText embeddings

In [ ]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        vec = list(map(float, tokens[1:]))
        vec = np.array(vec)
        data[tokens[0]] = vec
    return data

In [ ]:
ft_vecs = load_vectors("/data/alolbuhtijarov/wiki-news-300d-1M-subword.vec")

In [ ]:
vocab_token_vectors = torch.FloatTensor([
    ft_vecs.get(w) if w in ft_vecs else np.random.rand(300) * 1e-3 for w in tokens
])

vocab_token_vectors.shape

In [ ]:
class SmallEncoder(nn.Module):
    def __init__(self, n_tokens=len(tokens), n_cat_features=len(categorical_vectorizer.vocabulary_), hid_size=128,
                freeze_embed=True):
        super().__init__()
        
        self.embed = nn.Embedding.from_pretrained(vocab_token_vectors, freeze=freeze_embed)

        self.layer_title = nn.Sequential(
            nn.Conv1d(in_channels=300, out_channels=300, kernel_size=3),
            nn.AdaptiveAvgPool1d(output_size=1),
            nn.BatchNorm1d(num_features=300),
            nn.ReLU(),
        )

        self.layer_text = nn.Sequential(
            nn.Conv1d(in_channels=300, out_channels=300, kernel_size=3),
            nn.AdaptiveAvgPool1d(output_size=1),
            nn.BatchNorm1d(num_features=300),
            nn.ReLU()
        )

        self.layer_cat = nn.Sequential(
            nn.Linear(n_cat_features, hid_size * 2),
            nn.BatchNorm1d(num_features=hid_size * 2),
            nn.ReLU(),
            nn.Linear(hid_size * 2, hid_size),
            nn.BatchNorm1d(num_features=hid_size),
            nn.ReLU()
        )

        self.predictor = nn.Linear(300 * 2 + hid_size, 1)


    def forward(self, batch):
        title = self.embed(batch["Title"])
        title = title.permute(0, 2, 1)
        title = self.layer_title(title).squeeze(-1)

        text = self.embed(batch["FullDescription"])
        text = text.permute(0, 2, 1)
        text = self.layer_text(text).squeeze(-1)

        cat = self.layer_cat(batch["Categorical"])

        x = torch.cat([title, text, cat], dim=1)
        return self.predictor(x).squeeze(-1)

https://pytorch.org/docs/stable/optim.html#per-parameter-options

In [ ]:
def separate_optimizer(net):
    embed_param = [kv[1] for kv in net.named_parameters() if kv[0] == 'embed.weight']
    model_params = [kv[1] for kv in net.named_parameters() if kv[0] != 'embed.weight']
    opt = torch.optim.Adam([
                {'params': model_params},
                {'params': embed_param, 'lr': 3e-4}
    ], lr=3e-3)
    return opt